<a href="https://colab.research.google.com/github/aslesani/pgmpy_fork/blob/master/src/default_test/imdb_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''Trains an LSTM model on the IMDB sentiment classification task.
The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
# Notes
- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.
- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
import numpy as np

from read_write import data_preparation_for_sequences_based_deep_models, convert_binary_classes_to_zero_and_one

Using TensorFlow backend.


In [2]:
import matplotlib.pyplot as plt

def plot_train_val_graph(history):
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = range(1, len(loss) + 1)
  print('epochs:' , epochs)
  plt.figure()
  plt.plot(epochs, loss, 'bo', label='Training loss')
  plt.plot(epochs, val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()
  plt.show()


In [4]:
def get_max_len_of_sequences(list_of_sequences):
  lengths = [len(list_of_sequences[i]) for i in range(len(list_of_sequences))]
  return max(lengths) , min(lengths) , lengths

In [5]:
def get_set_of_sensor_events(list_of_sequences):
 
  set_of_sensor_events = set()
  
  for i in range(len(list_of_sequences)):
      set_of_sensor_events = set_of_sensor_events.union(set(list_of_sequences[i]))
  
  return set_of_sensor_events, len(set_of_sensor_events)
  

In [5]:
#!git clone https://github.com/aslesani/pgmpy_fork.git
#ls
#!git clone https://github.com/aslesani/created_dataset.git
#!rm -r pgmpy_fork  
#cd pgmpy_fork/src/default_test
#cd ..

In [5]:
from keras import backend as K

def mcor(y_true, y_pred):
    #matthews_correlation
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos
 
 
    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos
 
 
    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)
 
 
    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)
 
 
    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
 
 
    return numerator / (denominator + K.epsilon())


def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [7]:
#!pip install tabulate 

In [3]:
from tabulate import tabulate

#print(tabulate([['Alice', 24], ['Bob', 19]], headers=['algorithm', 'acc']))

def print_list_of_lists(data , headers):
    print(tabulate(data, headers=headers))



In [7]:
def test_print_list_of_lists():
    data = [['Alice', 24], ['Bob', 19]]
    headers=['algorithm', 'acc']
    print_list_of_lists(data , headers)


In [8]:
test_print_list_of_lists()

algorithm      acc
-----------  -----
Alice           24
Bob             19


In [9]:
def imdb_lstm_data_preparation(max_features = 20000, maxlen = 80):
  #max_features = 20000#number_of_events
  # cut texts after this number of words (among top max_features most common words)
  #maxlen = 10#max_seq_len

  print('Loading data...')
  (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
  print(len(x_train), 'train sequences')
  print(len(x_test), 'test sequences')

  #print('before apply pad_sequence, x_train[0]:' , x_train[0])

  print('Pad sequences (samples x time)')
  x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
  x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
  print('x_train shape:', x_train.shape)
  print('x_test shape:', x_test.shape)
  
  return x_train, x_test, y_train, y_test, max_features, maxlen

In [12]:
#! cd pgmpy_fork/src/default_test
#!ls
#!git clone https://github.com/pgmpy/pgmpy 
#cd ..
#!ls
#!cd pgmpy/
#pip install -r requirements.txt
#!python setup.py install


In [7]:
def make_lstm_without_embedding(x_train, y_train,x_test, y_test, max_features,embedding_vector_dim = 64, batch_size = 32, epochs = 5, 
                                   loss = 'binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'], plot_train_val_graph = False):
  
  #batch_size = 32

  print('Build model...')
  model = Sequential()
  #model.add(Embedding(max_features+1, embedding_vector_dim))
  #model.add(SimpleRNN(64, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
  #model.add(SimpleRNN(64, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
  model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(1, activation='sigmoid'))

  model.summary()

  # try using different optimizers and different optimizer configs
  model.compile(loss= loss,
                optimizer=optimizer,
                metrics= metrics)#, mcor,recall, f1])

  print('Train...')
  history = model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test))
  score, acc = model.evaluate(x_test, y_test,
                              batch_size=batch_size)
  print('Test score:', score)# i think score is loss value
  print('Test accuracy:', acc)
 
  if plot_train_val_graph:
      plot_train_val_graph(history)
      
  return score, acc, history, len(x_train), len(x_test)

In [4]:
def create_model_and_apply_on_data(x_train, y_train,x_test, y_test, max_features,
                                   embedding_vector_dim = 64, batch_size = 32, epochs = 10, 
                                   loss = 'binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'], 
                                   plot_train_val_graph = False,
                                   number_of_lstm_layers = 1,
                                   ID_of_layer_to_repeat = 0):
  '''
  Parameters:
  ===============
  number_of_lstm_layers (default value = 1)
      indicate the number of layers in stack of layers
  
  ID_of_layer_to_repeat (default value = 0)
     0: LSTM
     1:RNN
     2: GRU
  
  '''
  #batch_size = 32

  print('Build model...')
  model = Sequential()
  model.add(Embedding(max_features+1, embedding_vector_dim))
  #model.add(SimpleRNN(64, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
  #model.add(SimpleRNN(64, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
  
  layer_names = [LSTM, SimpleRNN, GRU]
  if number_of_lstm_layers > 1:
        for l in range(number_of_lstm_layers-1):
            model.add(layer_names[ID_of_layer_to_repeat](64, dropout=0.2, recurrent_dropout=0.2, return_sequences = True))
  model.add(layer_names[ID_of_layer_to_repeat](64, dropout=0.2, recurrent_dropout=0.2))
        
  model.add(Dense(1, activation='sigmoid'))

  model.summary()

  # try using different optimizers and different optimizer configs
  model.compile(loss= loss,
                optimizer=optimizer,
                metrics= metrics)#, mcor,recall, f1])

  print('Train...')
  history = model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test))
  score, acc = model.evaluate(x_test, y_test,
                              batch_size=batch_size)
  #print('Test score:', score)# i think score is loss value
  #print('Test accuracy:', acc)
 
  #print(model.layers[0].output)
  if plot_train_val_graph:
      plot_train_val_graph(history)
      
  return score, acc, history, len(x_train), len(x_test)

In [9]:
def select_hyperparameters(address_to_read, ID_of_layer_to_repeat):
    #address_to_read= r"E:/pgmpy/Seq of sensor events_based on activities/based_on_activities.csv"
    #address_to_read = r"E:\pgmpy\Seq of sensor events_no overlap_based on different deltas\delta_{}min.csv"
    #address_to_read = r"E:\pgmpy\Seq of sensor events_based_on_activity_and_no_overlap_delta\delta_{}min.csv"
    
    for layers in range(2,5):
        print("number_of_lstm_layers:" , layers)

        results = []
        for delta in list(range(1,10)):# + [30,45,60,75,90,100]:#, 120,150, 180,200,240,300,400,500,600,700,800,900,1000]: #:
            print("delta:" , delta)
            x_train, x_test, y_train, y_test, max_features, maxlen = data_preparation_for_sequences_based_deep_models(address_to_read = address_to_read.format(delta),
                                                                                                                  number_of_words = 122, 
                                                                                                                  max_seq_len = 20)
            test_score, test_acc, history, num_of_train_samples, num_of_test_sample = create_model_and_apply_on_data(x_train =x_train, 
                                                                                                y_train = y_train,
                                                                                                x_test = x_test, 
                                                                                                y_test = y_test, 
                                                                                                max_features = max_features,
                                                                                                number_of_lstm_layers = layers,
                                                                                                ID_of_layer_to_repeat = ID_of_layer_to_repeat )
            results.append([delta, 
                        num_of_train_samples, 
                        num_of_test_sample, 
                        history.history['loss'][-1], 
                        history.history['acc'][-1] ,
                        test_score, 
                        test_acc])#, history.history
        #print(history.history)
        print_list_of_lists(results, ['delta(min)' ,
                                  '#train', 
                                  '#test', 
                                  'train loss(last)', 
                                  'train_acc(last)', 
                                  'val loss ', 
                                  'val acc', ])#'history'
        print("results:", results)
        #print("results[:][-1]:", results[:][-1])
        #best_val_acc_index = np.argmax(results[:][-1])
        #print("****************************************")
        #print("best vlidation acc delta:" , results[best_val_acc_index][0])
    
    return results
    
    

In [ ]:
#address_to_read= r"E:/pgmpy\Twor2009\Seq of sensor events_based on activities\based_on_activities.csv"
address_to_read = r"E:\pgmpy\Twor2009\Seq of sensor events_no overlap_based on different deltas\delta_{}min.csv"
#address_to_read = r"E:\pgmpy\Twor2009\Seq of sensor events_based_on_activity_and_no_overlap_delta\delta_{}min.csv"
print(address_to_read)
ID_of_layer_to_repeat = 0
print("ID_of_layer_to_repeat:" , ID_of_layer_to_repeat)
results = select_hyperparameters(address_to_read = address_to_read , ID_of_layer_to_repeat = ID_of_layer_to_repeat)

E:\pgmpy\Twor2009\Seq of sensor events_no overlap_based on different deltas\delta_{}min.csv
ID_of_layer_to_repeat: 0
number_of_lstm_layers: 2
delta: 1
Pad sequences (samples x time)
x_train shape: (11618, 20)
x_test shape: (2904, 20)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_33 (Embedding)     (None, None, 64)          7872      
_________________________________________________________________
lstm_90 (LSTM)               (None, None, 64)          33024     
_________________________________________________________________
lstm_91 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 65        
Total params: 73,985
Trainable params: 73,985
Non-trainable params: 0
_________________________________________________________________
Train...
Train on 116

Train...
Train on 5078 samples, validate on 1269 samples
Epoch 1/10
5078/5078 [==============================] - 82s 16ms/step - loss: 0.2358 - acc: 0.9080 - val_loss: 0.1625 - val_acc: 0.9409
Epoch 2/10
5078/5078 [==============================] - 13s 3ms/step - loss: 0.1494 - acc: 0.9427 - val_loss: 0.1429 - val_acc: 0.9645
Epoch 3/10
5078/5078 [==============================] - 14s 3ms/step - loss: 0.1397 - acc: 0.9447 - val_loss: 0.1826 - val_acc: 0.9346
Epoch 4/10
5078/5078 [==============================] - 14s 3ms/step - loss: 0.1350 - acc: 0.9462 - val_loss: 0.1626 - val_acc: 0.9441
Epoch 5/10
5078/5078 [==============================] - 14s 3ms/step - loss: 0.1333 - acc: 0.9484 - val_loss: 0.1546 - val_acc: 0.9401
Epoch 6/10
5078/5078 [==============================] - 14s 3ms/step - loss: 0.1323 - acc: 0.9478 - val_loss: 0.1594 - val_acc: 0.9496
Epoch 7/10
5078/5078 [==============================] - 14s 3ms/step - loss: 0.1302 - acc: 0.9502 - val_loss: 0.1626 - val_acc: 0.93

3573/3573 [==============================] - 10s 3ms/step - loss: 0.1282 - acc: 0.9530 - val_loss: 0.2263 - val_acc: 0.9115
Epoch 8/10
3573/3573 [==============================] - 10s 3ms/step - loss: 0.1265 - acc: 0.9524 - val_loss: 0.1597 - val_acc: 0.9451
Epoch 9/10
3573/3573 [==============================] - 10s 3ms/step - loss: 0.1257 - acc: 0.9527 - val_loss: 0.1611 - val_acc: 0.9462
Epoch 10/10
893/893 [==============================] - 1s 682us/step
delta: 8
Pad sequences (samples x time)
x_train shape: (3275, 20)
x_test shape: (818, 20)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_40 (Embedding)     (None, None, 64)          7872      
_________________________________________________________________
lstm_104 (LSTM)              (None, None, 64)          33024     
_________________________________________________________________
lstm_105 (LSTM)              (None, 

Train...
Train on 11618 samples, validate on 2904 samples
Epoch 1/10
11618/11618 [==============================] - 882s 76ms/step - loss: 0.2319 - acc: 0.9046 - val_loss: 0.3447 - val_acc: 0.8826
Epoch 2/10
11618/11618 [==============================] - 57s 5ms/step - loss: 0.1622 - acc: 0.9340 - val_loss: 0.2010 - val_acc: 0.9101
Epoch 3/10
11618/11618 [==============================] - 57s 5ms/step - loss: 0.1565 - acc: 0.9346 - val_loss: 0.2378 - val_acc: 0.8994
Epoch 4/10
11618/11618 [==============================] - 61s 5ms/step - loss: 0.1545 - acc: 0.9378 - val_loss: 0.2131 - val_acc: 0.9060
Epoch 5/10
11618/11618 [==============================] - 61s 5ms/step - loss: 0.1528 - acc: 0.9381 - val_loss: 0.2072 - val_acc: 0.9077
Epoch 6/10
11618/11618 [==============================] - 61s 5ms/step - loss: 0.1514 - acc: 0.9402 - val_loss: 0.1715 - val_acc: 0.9456
Epoch 7/10
11618/11618 [==============================] - 60s 5ms/step - loss: 0.1486 - acc: 0.9406 - val_loss: 0.1842

5078/5078 [==============================] - 25s 5ms/step - loss: 0.1377 - acc: 0.9472 - val_loss: 0.1699 - val_acc: 0.9535
Epoch 5/10
5078/5078 [==============================] - 25s 5ms/step - loss: 0.1356 - acc: 0.9480 - val_loss: 0.1560 - val_acc: 0.9417
Epoch 6/10
5078/5078 [==============================] - 25s 5ms/step - loss: 0.1357 - acc: 0.9478 - val_loss: 0.1387 - val_acc: 0.9504
Epoch 7/10
5078/5078 [==============================] - 25s 5ms/step - loss: 0.1341 - acc: 0.9496 - val_loss: 0.1519 - val_acc: 0.9590
Epoch 8/10
5078/5078 [==============================] - 26s 5ms/step - loss: 0.1323 - acc: 0.9516 - val_loss: 0.1652 - val_acc: 0.9464
Epoch 9/10
5078/5078 [==============================] - 25s 5ms/step - loss: 0.1304 - acc: 0.9488 - val_loss: 0.1417 - val_acc: 0.9433
Epoch 10/10
1269/1269 [==============================] - 2s 1ms/step
delta: 5
Pad sequences (samples x time)
x_train shape: (4432, 20)
x_test shape: (1107, 20)
Build model...
__________________________

...

7737/7737 [==============================] - 348s 45ms/step - loss: 0.2551 - acc: 0.8949 - val_loss: 0.1868 - val_acc: 0.9271
Epoch 2/10
7737/7737 [==============================] - 55s 7ms/step - loss: 0.1557 - acc: 0.9407 - val_loss: 0.1828 - val_acc: 0.9126
Epoch 3/10
7737/7737 [==============================] - 57s 7ms/step - loss: 0.1477 - acc: 0.9446 - val_loss: 0.1953 - val_acc: 0.9250
Epoch 4/10
7737/7737 [==============================] - 58s 8ms/step - loss: 0.1461 - acc: 0.9446 - val_loss: 0.1879 - val_acc: 0.9224
Epoch 5/10
7737/7737 [==============================] - 58s 7ms/step - loss: 0.1439 - acc: 0.9458 - val_loss: 0.1745 - val_acc: 0.9255
Epoch 6/10
7737/7737 [==============================] - 57s 7ms/step - loss: 0.1427 - acc: 0.9467 - val_loss: 0.1748 - val_acc: 0.9328
Epoch 7/10
7737/7737 [==============================] - 58s 7ms/step - loss: 0.1410 - acc: 0.9473 - val_loss: 0.1788 - val_acc: 0.9338
Epoch 8/10
7737/7737 [==============================] - 58s 7ms/

c:\python37_64\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.873560). Check your callbacks.
  % delta_t_median)


6047/6047 [==============================] - 1114s 184ms/step - loss: 0.2621 - acc: 0.8909 - val_loss: 0.1717 - val_acc: 0.9265
Epoch 2/10
6047/6047 [==============================] - 50s 8ms/step - loss: 0.1549 - acc: 0.9390 - val_loss: 0.1609 - val_acc: 0.9351
Epoch 3/10
6047/6047 [==============================] - 47s 8ms/step - loss: 0.1468 - acc: 0.9415 - val_loss: 0.1906 - val_acc: 0.9477
Epoch 4/10
6047/6047 [==============================] - 51s 8ms/step - loss: 0.1437 - acc: 0.9448 - val_loss: 0.1642 - val_acc: 0.9418
Epoch 5/10
6047/6047 [==============================] - 51s 8ms/step - loss: 0.1443 - acc: 0.9458 - val_loss: 0.1552 - val_acc: 0.9576
Epoch 6/10
6047/6047 [==============================] - 51s 8ms/step - loss: 0.1393 - acc: 0.9454 - val_loss: 0.1440 - val_acc: 0.9338
Epoch 7/10
6047/6047 [==============================] - 51s 8ms/step - loss: 0.1394 - acc: 0.9456 - val_loss: 0.1822 - val_acc: 0.9351
Epoch 8/10
6047/6047 [==============================] - 51s 8m

c:\python37_64\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.565105). Check your callbacks.
  % delta_t_median)


4432/4432 [==============================] - 524s 118ms/step - loss: 0.2877 - acc: 0.8705 - val_loss: 0.1385 - val_acc: 0.9503
Epoch 2/10
2880/4432 [==================>...........] - ETA: 12s - loss: 0.1506 - acc: 0.9434

In [19]:
print(results[:][-1])
print(np.argmax(results[:][-1]))

TypeError: 'NoneType' object is not subscriptable

In [22]:
address_to_read= r"E:/pgmpy/Twor2009/Seq of sensor events_based on activities/based_on_activities.csv"

x_train, x_test, y_train, y_test, max_features, maxlen = data_preparation_for_sequences_based_deep_models(address_to_read)

test_score, test_acc, history, num_of_train_samples, num_of_test_sample = create_model_and_apply_on_data(x_train, 
                                                                                                        y_train,
                                                                                                        x_test, 
                                                                                                        y_test, 
                                                                                                        max_features,
                                                                                                        ID_of_layer_to_repeat = ID_of_layer_to_repeat)
print("test_score:",test_score)
print("test_acc:" , test_acc)
print("history:" , history.history)
print("num_of_train_samples:" , num_of_train_samples)
print("num_of_test_sample:" , num_of_test_sample)


Pad sequences (samples x time)
x_train shape: (2578, 20)
x_test shape: (644, 20)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_104 (Embedding)    (None, None, 64)          7808      
_________________________________________________________________
gru_69 (GRU)                 (None, 64)                24768     
_________________________________________________________________
dense_104 (Dense)            (None, 1)                 65        
Total params: 32,641
Trainable params: 32,641
Non-trainable params: 0
_________________________________________________________________
Train...
Train on 2578 samples, validate on 644 samples
Epoch 1/5
2578/2578 [==============================] - 68s 26ms/step - loss: 0.2978 - acc: 0.8798 - val_loss: 0.8843 - val_acc: 0.4953
Epoch 2/5
2578/2578 [==============================] - 5s 2ms/step - loss: 0.1502 - acc: 0.9298 - val_loss: 0.7980 

In [16]:
x_train, x_test, y_train, y_test, max_features, maxlen = imdb_lstm_data_preparation(maxlen=10)
#my_x_train, my_x_test, my_y_train, my_y_test, my_max_features, my_maxlen = data_preparation_for_sequences_based_deep_models(address_to_read)
#x_train, x_test, y_train, y_test, max_features, maxlen = data_preparation_for_sequences_based_deep_models(address_to_read)#imdb_lstm_data_preparation(maxlen=10)
create_model_and_apply_on_data(x_train, y_train,x_test, y_test, max_features,embedding_vector_dim = maxlen)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 10)
x_test shape: (25000, 10)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 10)          200010    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                19200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 219,275
Trainable params: 219,275
Non-trainable params: 0
_________________________________________________________________
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 16s 653us/step - loss: 0.6015 - acc: 0.6613 - val_loss: 0.5451 - val_acc: 0.7164
Epoch 2/5
25000/25000 [======================

(0.52888289176940917,
 0.72951999999999995,
 25000,
 25000)

In [ ]:
embeddings = model.layers[0].get_weights()[0]
print(embeddings)

In [ ]:
# `word_to_index` is a mapping (i.e. dict) from words to their index, e.g. `love`: 69
words_embeddings = {w:embeddings[idx] for w, idx in word_to_index.items()}

# now you can use it like this for example
print(words_embeddings['love'])  # possible output: [0.21, 0.56, ..., 0.65, 0.10]


In [ ]:
print(type(score) , type(acc))